In [1]:
pdf_path = "data/Сборка шкаф-купе.pdf"

## PDF Preprocessing

In [ ]:
import os
import uuid
import pdfplumber

def extract_pdf(pdf_path, mode="mixed"):
    """
    Извлечение контента из PDF.

    mode:
        - "only_text" : только текст со страниц
        - "images"    : рендер каждой страницы в картинку
        - "mixed"     : текст + картинки со страниц
    """
    all_content = []
    pdf_id = str(uuid.uuid4())
    os.makedirs(f"images/{pdf_id}", exist_ok=True)

    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):

            if mode == "only_text":
                text = page.extract_text()
                if text:
                    all_content.append(text.strip())

            elif mode == "images":
                # Сохраняем целую страницу как картинку
                im = page.to_image(resolution=300)
                img_filename = f"images/{pdf_id}/page{page_num}.png"
                im.save(img_filename, format="PNG")
                all_content.append(f"[image]('{img_filename}')")

            elif mode == "mixed":
                # 1. Текст
                text = page.extract_text()
                if text:
                    all_content.append(text.strip())
                
                # 2. Картинки
                for img_index, img in enumerate(page.images, start=1):
                    x0, top, x1, bottom = img["x0"], img["top"], img["x1"], img["bottom"]
                    im = page.crop((x0, top, x1, bottom)).to_image(resolution=300)
                    
                    img_filename = f"images/{pdf_id}/page{page_num}_img{img_index}.png"
                    im.save(img_filename, format="PNG")
                    all_content.append(f"[image]('{img_filename}')")

    text = "\n\n".join(all_content)
    return pdf_id, text



# пример вызова
pdf_id, text = extract_pdf(pdf_path, 'images')
image_paths = [f"images/{pdf_id}/{image_name}" for image_name in os.listdir(f"images/{pdf_id}")]
print(pdf_id)
print(text)

7b9dbd7c-b16d-490c-9ce4-a97d620c0470
«Гамма»
Шкаф купе
1598 2100 520
«Gamma»
Sliding wardrobe
www.stendmebel.ru
RU Инструкция по сборке / EN Assembling instruction / ES Montaje de instrucción / CZ Montážní návod
e-mail: stendmebel@mail.ru DE Montageanleitung / FR Notice de montage / LV Montāžas instrukcijas / PL Instrukcje składania / SK Montážny návod

RU FR PL CZ
ВНИМАНИЕ! ATTENTION! UWAGA! POZORNOST!
Важная информация. Внимательно изучите её перед Informations importantes. Examinez-les attentivement Ważna informacja. Przestudiuj to dokładnie zanim działanie produktu. Důležitá informace. Předtím si to pečlivě prostudujte
эксплуатацией изделия. avant d'utiliser l'article. Używaj produktu tylko zgodnie z jego przeznaczeniem. provoz výrobku.
• Используйте изделие только по прямому назначению. • Utilisez l'article uniquement aux fins prévues. Złóż produkt tylko w całości Používejte výrobek pouze k určenému účelu.
• Сборку изделия производите только в полном • Assemblez l'article uniqueme

## Image-to-text

### Image-text-to-text

In [3]:
# Инициализация экземпляра PaddleOCR
from paddleocr import PaddleOCR
ocr = PaddleOCR(lang="ru", use_angle_cls=False)

def ocr_image(image_path):
    result = ocr.predict(input=image_path)
    text = '\n'.join(result[0]['rec_texts'])
    return text

C:\Users\Sefixnep\AppData\Local\Temp\ipykernel_35940\1253384450.py:3: DeprecationWarning: The parameter `use_angle_cls` has been deprecated and will be removed in the future. Please use `use_textline_orientation` instead.
  ocr = PaddleOCR(lang="ru", use_angle_cls=False)
c:\Users\Sefixnep\AppData\Local\Programs\Python\Python311\Lib\site-packages\paddle\utils\cpp_extension\extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\Sefixnep\.paddlex\official_models\PP-LCNet_x1_0_doc_ori`.
Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\Sefixnep\.p

In [4]:
import numpy as np
from tqdm import tqdm

texts = [ocr_image(image_path) for image_path in tqdm(image_paths)]

100%|██████████| 4/4 [00:05<00:00,  1.27s/it]


In [5]:
len_threshold = 25
no_text_image_idxs = np.where(np.array(list(map(len, texts))) < len_threshold)[0].tolist()
print(no_text_image_idxs)

[0, 1, 2, 3]


In [6]:
del ocr # clear memory

### Image to text

In [ ]:
import torch
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

device = "cuda" if torch.cuda.is_available() else "cpu"

# Можно заменить на blip-image-captioning-large
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to(device)

def get_image_description(image_path):
    img = Image.open(image_path).convert("RGB")

    # Более информативный запрос
    text = ("Describe this image in extreme detail. "
            "List all visible objects, their colors, shapes, positions, background elements, "
            "and any possible actions or relationships.")

    inputs = processor(img, text, return_tensors="pt").to(device)

    out = model.generate(
        **inputs,
        max_new_tokens=200,        # длина описания
        num_beams=5,              # beam search для качества
        length_penalty=1.5,       # поощряет длинные описания
        repetition_penalty=2.0,   # избегаем повторов
        no_repeat_ngram_size=3,   # не повторять одинаковые фразы
        early_stopping=True
    )

    return processor.decode(out[0], skip_special_tokens=True)[len(text):].strip()

In [ ]:
for image_idx in tqdm(no_text_image_idxs):
    texts[image_idx] = get_image_description(image_paths[image_idx])

In [ ]:
texts

In [ ]:
del processor, model # clear memory

### Replace text

In [ ]:
for i in range(len(image_paths)):
    text = text.replace(f"[image]('{image_paths[i]}')", f"Image: {texts[i]}")

## PostProcessing text

In [ ]:
prompt = f'''
You are a professional comic storyboard writer. 
Your task is to transform a given long text (possibly from parsed PDF documents with mixed text and image captions) into a cinematic comic script. 

Instructions:
1. Extract the main ideas and convert them into a sequence of **comic pages**, each page having exactly **4 panels** (so every page = 1 scene = 4 panels).
2. Write everything in **English**.
3. Output must be strictly in the following format (Python tuple):
(
    "Unified cinematic style for the entire series: [one single descriptive paragraph describing art style, characters, settings, rendering details, coloring, film look, and atmosphere that apply to the entire comic].",
    [
        "COMIC_SCENE_01_NAME — 4-panel comic page. Panel 1: [description]. Bubble: [dialogue]. Panel 2: [description]. Bubble: [dialogue]. Panel 3: [description]. Bubble: [dialogue]. Panel 4: [description]. Bubble: [dialogue].",
        "COMIC_SCENE_02_NAME — 4-panel comic page, direct continuation. Panel 1: [description]. Bubble: [dialogue]. Panel 2: [description]. Bubble: [dialogue]. Panel 3: [description]. Bubble: [dialogue]. Panel 4: [description]. Bubble: [dialogue].",
        ...
    ]
)

Rules:
- Do not include any extra variables or explanations (no `unified_style_en = ...`, no `slides = ...`). Only return the tuple above.
- Replace references to Russian legal text, laws, and formal descriptions with **visual narrative and natural dialogue** between characters.
- Incorporate image captions (e.g., “a photography of a judge’s gavel”) as panel **visual elements**, adapted into cinematic descriptions.
- Ensure dialogue is simple, conversational, and in English.
- Maintain continuity: the same characters should appear across panels and scenes, with consistent roles and personalities.
- Each scene = exactly one 4-panel comic page.
- Maximum 3 comic scenes per run (12 panels total).
- Output must be valid Python (tuple of a string + list of strings), parsable with `ast.literal_eval`.

---
SOURCE TEXT:
{text}
'''

In [ ]:
import os
from typing import List, Dict, Optional
from openai import OpenAI


class OpenRouterChatClient:
    def __init__(
        self,
        api_key: Optional[str] = None,
        base_url: str = "https://openrouter.ai/api/v1",
        http_referer: Optional[str] = None,
        x_title: Optional[str] = None,
    ) -> None:
        key = api_key or os.getenv("OPENROUTER_API_KEY")
        if not key:
            raise RuntimeError("OPENROUTER_API_KEY is not set")

        extra_headers: Dict[str, str] = {}
        http_referer = http_referer or os.getenv("OPENROUTER_HTTP_REFERER")
        x_title = x_title or os.getenv("OPENROUTER_X_TITLE")
        if http_referer:
            extra_headers["HTTP-Referer"] = http_referer
        if x_title:
            extra_headers["X-Title"] = x_title

        self.client = OpenAI(
            base_url=base_url,
            api_key=key,
            default_headers=extra_headers or None,
        )

    def chat(
        self,
        model: str,
        messages: List[Dict[str, str]],
        temperature: Optional[float] = None,
        max_tokens: Optional[int] = None,
        extra_body: Optional[Dict] = None,
    ) -> str:
        body = extra_body or {}
        if temperature is not None:
            body["temperature"] = temperature
        if max_tokens is not None:
            body["max_tokens"] = max_tokens

        completion = self.client.chat.completions.create(
            model=model,
            messages=messages,
            extra_body=body,
        )
        # Normalize to text output
        choice = completion.choices[0]
        content = getattr(choice.message, "content", None)
        if isinstance(content, list):
            # Join text blocks if segmented
            parts = []
            for block in content:
                if isinstance(block, dict) and block.get("type") in ("text", "output_text"):
                    val = block.get("text") or block.get("content")
                    if isinstance(val, str):
                        parts.append(val)
                elif isinstance(block, str):
                    parts.append(block)
            return "\n".join(parts).strip()
        if isinstance(content, str):
            return content
        return str(content)

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
import ast
import re

client = OpenRouterChatClient(api_key=os.getenv("text_api"))
def extract_comic_tuple(text: str):
    """
    Извлекает первый валидный Python-кортеж из ответа модели,
    даже если вокруг есть лишний текст.
    """
    # Находим участок от первой открывающей "(" до последней закрывающей ")"
    match = re.search(r"\(.*\)", text, re.DOTALL)
    if not match:
        raise ValueError("Не найден кортеж в ответе модели.")
    
    tuple_str = match.group(0)

    try:
        parsed = ast.literal_eval(tuple_str)
    except Exception as e:
        raise ValueError(f"Ошибка при парсинге кортежа: {e}\nИсходный текст:\n{tuple_str}")
    
    # Проверим, что структура правильная
    if not (isinstance(parsed, tuple) and len(parsed) == 2):
        raise ValueError("Найденный объект не является кортежем из 2 элементов.")
    if not isinstance(parsed[0], str):
        raise ValueError("Первый элемент кортежа должен быть строкой (unified_style).")
    if not isinstance(parsed[1], list):
        raise ValueError("Второй элемент кортежа должен быть списком (slides).")
    
    return parsed

while True:
    try:
        llm_output = client.chat(
            model="deepseek/deepseek-chat-v3.1",
            messages=[
                {"role": "user", "content": prompt},
            ],
            temperature=0.7,
        )
        unified_style, slides = extract_comic_tuple(llm_output)
    except Exception as e:
        continue
    else:
        break

text_slides = " ".join(slides)
print("Unified style:\n", unified_style)
print("\nSlides:")
for i, scene in enumerate(slides, 1):
    print(f"\nScene {i}:\n{scene}")


## Generate comics

In [ ]:
import os
import time
import json
import base64
import logging
import requests
from pathlib import Path
from openai import OpenAI


class OpenRouterImageAPI:
    def __init__(self, api_key, model=None, size=None):
        default_headers = {}
        if os.getenv("OPENROUTER_HTTP_REFERER"):
            default_headers["HTTP-Referer"] = os.getenv("OPENROUTER_HTTP_REFERER")
        if os.getenv("OPENROUTER_X_TITLE"):
            default_headers["X-Title"] = os.getenv("OPENROUTER_X_TITLE")

        self.client = OpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=api_key,
            default_headers=default_headers or None,
        )

        # Keep for manual HTTP fallback
        self.api_key = api_key
        self.default_headers = default_headers or None

        # Используй модель, которая поддерживает image output
        self.model = model or os.getenv("OPENROUTER_IMAGE_MODEL", "google/gemini-2.5-flash-image-preview")
        self.size = size or os.getenv("OPENROUTER_IMAGE_SIZE", "1024x1024")

    def generate_image(self, prompt, num_images=1, output_dir="outputs", prefix="slide"):
        Path(output_dir).mkdir(parents=True, exist_ok=True)
        logging.info(f"Generating image via OpenRouter model='{self.model}' size='{self.size}'")

        resp = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            extra_body={
                "modalities": ["image", "text"],
                "n": num_images,
                "size": self.size,
                "response_format": "b64_json"
            }
        )

        saved_paths = []
        timestamp = int(time.time())

        image_counter = 0
        for i, choice in enumerate(resp.choices, start=1):
            content = getattr(choice.message, "content", None)
            if not content:
                logging.error(f"No content in choice: {choice}")
                continue

            # Normalize to iterable of blocks
            blocks = content if isinstance(content, list) else [content]

            for block in blocks:
                b64 = None
                url_to_fetch = None

                if isinstance(block, dict):
                    btype = block.get("type") or block.get("kind")
                    # OpenRouter 'output_image' (most common)
                    if btype in ("output_image", "image"):
                        b64 = block.get("image_base64") or block.get("b64_json")
                        # Some providers nest it
                        if not b64 and isinstance(block.get("image"), dict):
                            b64 = block["image"].get("b64_json") or block["image"].get("base64")
                    # Our previous handler for data URLs
                    if btype == "image_url" and isinstance(block.get("image_url"), dict):
                        img_url = block["image_url"].get("url", "")
                        if isinstance(img_url, str) and img_url.startswith("data:"):
                            try:
                                _, b64 = img_url.split(",", 1)
                            except Exception:
                                b64 = None
                        elif isinstance(img_url, str) and img_url.startswith("http"):
                            url_to_fetch = img_url
                elif isinstance(block, str):
                    text = block.strip()
                    if text.startswith("data:"):
                        try:
                            _, b64 = text.split(",", 1)
                        except Exception:
                            b64 = None
                    else:
                        # Heuristic: treat long strings as raw base64
                        if len(text) > 100:
                            b64 = text

                # Save from base64 if available
                if b64:
                    try:
                        image_bytes = base64.b64decode(b64)
                        image_counter += 1
                        file_path = Path(output_dir) / f"{prefix}-{timestamp}-{image_counter}.png"
                        with open(file_path, "wb") as f:
                            f.write(image_bytes)
                        saved_paths.append(str(file_path))
                    except Exception as e:
                        logging.error(f"Failed to decode/save base64 image: {e}")
                        continue

                # Fallback: fetch via URL if provided
                if url_to_fetch:
                    try:
                        r = requests.get(url_to_fetch, timeout=60)
                        r.raise_for_status()
                        image_counter += 1
                        file_path = Path(output_dir) / f"{prefix}-{timestamp}-{image_counter}.png"
                        with open(file_path, "wb") as f:
                            f.write(r.content)
                        saved_paths.append(str(file_path))
                    except Exception as e:
                        logging.error(f"Failed to download image from URL: {e}")
                        continue

        if not saved_paths:
            # Deep fallback: scan the entire response object for base64 image strings
            try:
                def walk_collect_b64(value, out_list):
                    if isinstance(value, dict):
                        for k, v in value.items():
                            # Common keys used by providers
                            if k in ("image_base64", "b64_json", "imageBytes", "image_base64_png") and isinstance(v, str):
                                out_list.append(v)
                            # Any URL fields containing data URLs
                            if k in ("url", "image_url") and isinstance(v, str) and v.startswith("data:"):
                                try:
                                    _, b = v.split(",", 1)
                                    out_list.append(b)
                                except Exception:
                                    pass
                            else:
                                walk_collect_b64(v, out_list)
                    elif isinstance(value, list):
                        for item in value:
                            walk_collect_b64(item, out_list)
                    elif isinstance(value, str):
                        text = value.strip()
                        if text.startswith("data:") and "," in text:
                            try:
                                _, b = text.split(",", 1)
                                out_list.append(b)
                            except Exception:
                                pass
                        else:
                            # Heuristic: very long base64-ish strings
                            if len(text) > 500 and all(c.isalnum() or c in "+/=\n\r" for c in text[:100]):
                                out_list.append(text)

                raw_obj = None
                try:
                    raw_obj = resp.model_dump()
                except Exception:
                    try:
                        raw_json = getattr(resp, "json", None)
                        if callable(raw_json):
                            raw_obj = raw_json()
                        else:
                            raw_obj = json.loads(str(resp))
                    except Exception:
                        raw_obj = None

                candidates = []
                if raw_obj is not None:
                    walk_collect_b64(raw_obj, candidates)

                for idx, b64 in enumerate(candidates, start=1):
                    try:
                        image_bytes = base64.b64decode(b64)
                        file_path = Path(output_dir) / f"{prefix}-{timestamp}-fallback-{idx}.png"
                        with open(file_path, "wb") as f:
                            f.write(image_bytes)
                        saved_paths.append(str(file_path))
                    except Exception:
                        continue
            except Exception as e:
                logging.debug(f"Fallback scan error: {e}")

        # Final fallback: call OpenRouter directly via HTTP to avoid SDK type quirks
        if not saved_paths:
            try:
                headers = {
                    "Authorization": f"Bearer {self.api_key}",
                    "Content-Type": "application/json",
                }
                if self.default_headers:
                    headers.update(self.default_headers)
                body = {
                    "model": self.model,
                    "messages": [{"role": "user", "content": prompt}],
                    # Provider-specific hints are forwarded by OpenRouter
                    "modalities": ["image", "text"],
                    "n": num_images,
                    "size": self.size,
                    "response_format": "b64_json",
                }
                http_resp = requests.post(
                    "https://openrouter.ai/api/v1/chat/completions",
                    headers=headers,
                    json=body,
                    timeout=120,
                )
                http_resp.raise_for_status()
                data = http_resp.json()

                # Reuse collector
                candidates = []
                def walk_collect_b64_http(value, out_list):
                    if isinstance(value, dict):
                        for k, v in value.items():
                            if k in ("image_base64", "b64_json", "imageBytes", "image_base64_png") and isinstance(v, str):
                                out_list.append(v)
                            if k in ("url", "image_url") and isinstance(v, str) and v.startswith("data:"):
                                try:
                                    _, b = v.split(",", 1)
                                    out_list.append(b)
                                except Exception:
                                    pass
                            else:
                                walk_collect_b64_http(v, out_list)
                    elif isinstance(value, list):
                        for item in value:
                            walk_collect_b64_http(item, out_list)
                    elif isinstance(value, str):
                        text = value.strip()
                        if text.startswith("data:") and "," in text:
                            try:
                                _, b = text.split(",", 1)
                                out_list.append(b)
                            except Exception:
                                pass
                        elif len(text) > 500 and all(c.isalnum() or c in "+/=\n\r" for c in text[:100]):
                            out_list.append(text)

                walk_collect_b64_http(data, candidates)

                for idx, b64 in enumerate(candidates, start=1):
                    try:
                        image_bytes = base64.b64decode(b64)
                        file_path = Path(output_dir) / f"{prefix}-{timestamp}-http-{idx}.png"
                        with open(file_path, "wb") as f:
                            f.write(image_bytes)
                        saved_paths.append(str(file_path))
                    except Exception:
                        continue
            except Exception as e:
                logging.error(f"HTTP fallback failed: {e}")

        if not saved_paths:
            # Regex scan of any text representation for embedded base64
            try:
                import re
                def extract_and_save_from_text(text: str, tag: str):
                    local_saved = []
                    if not text:
                        return local_saved
                    # data URL pattern first
                    for m in re.finditer(r"data:image/[^;]+;base64,([A-Za-z0-9+/=\r\n]+)", text):
                        b64 = m.group(1)
                        try:
                            image_bytes = base64.b64decode(b64)
                            file_path = Path(output_dir) / f"{prefix}-{timestamp}-scan-{tag}-{len(local_saved)+1}.png"
                            with open(file_path, "wb") as f:
                                f.write(image_bytes)
                            local_saved.append(str(file_path))
                        except Exception:
                            continue
                    # Generic long base64 chunks (PNG/JPEG often start with iVBOR or /9j/)
                    for m in re.finditer(r"(iVBOR|/9j/)[A-Za-z0-9+/=\r\n]{400,}", text):
                        b64 = m.group(0)
                        try:
                            image_bytes = base64.b64decode(b64)
                            file_path = Path(output_dir) / f"{prefix}-{timestamp}-scan-{tag}-{len(local_saved)+1}.png"
                            with open(file_path, "wb") as f:
                                f.write(image_bytes)
                            local_saved.append(str(file_path))
                        except Exception:
                            continue
                    return local_saved

                text_repr = None
                try:
                    text_repr = resp.model_dump_json()  # may omit large blobs
                except Exception:
                    text_repr = str(resp)
                saved_paths.extend(extract_and_save_from_text(text_repr, "obj"))
            except Exception:
                pass

        if not saved_paths:
            raise Exception("Image generation returned no images")

        return saved_paths


image_api = os.getenv("image_api")
client = OpenRouterImageAPI(image_api, model="google/gemini-2.5-flash-image-preview")

Path(f"outputs/{pdf_id}").mkdir(parents=True, exist_ok=True)

full_prompt = f"{unified_style} {text_slides}"
try:
    saved = client.generate_image(full_prompt, num_images=1, output_dir=f"outputs/{pdf_id}", prefix=f"slide-{i}")
except Exception as e:
    pass